In [1]:
# CO 370 Group Project Code
# This codes reads in a CSV file, then performs the calculations

# Sources used:
# https://www.w3schools.com/python/pandas/pandas_csv.asp
# https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe 
# https://stackoverflow.com/questions/1541797/how-do-i-check-if-there-are-duplicates-in-a-flat-list
# https://docs.gurobi.com/projects/optimizer/en/current/reference/python/linexpr.html

In [2]:
# Imports
from gurobipy import *
import pandas as pd # Used to read csv files. 

In [3]:
# Variables to set
PRODUCT_CSV = "processed_optimization_data.csv" # This is the file name of the csv that contains all of the products along with their associated parameters
SUPPLEMENT_PRODUCT_CSV = "env_product_supplement.csv" # TODO: Look at combining this with the above
SHELF_CSV = "env_shelves.csv" # TODO: add description
NUMBER_LEVELS_PER_SHELF = 5 # This is the number of levels on each shelving
NUMBER_SLOTS_PER_SHELF = 50 # This is the number of product slots each shelf can hold
PREDICT_CUSTOMERS_PER_DAY = 100

In [4]:
# Variables set by script:
NUMBER_OF_SHELVES = 35 # Redefine in CSV TODO remove default

In [5]:
# Product class, used to define the parameters for each product
class product:
    def __init__(self):
        self.product_name = "" # This stores the name of each product
        self.shelf_type = "not set" # This is the shelf type, for example R, S or F
        self.is_essential_item = False # This determines if this product makes up high traffic route
        self.product_category = "" # This is a string with the product category, for example "fish"
        self.profit_per_unit = -1 # This is the profit made by selling 1 unit of the product
        self.prob_of_intentional_purchase = -1 # This is in [0,1] and indicates the probability that a customer intends to purchase this product
                                          # This is for example the probability that someone has the item on their shopping list and hence will buy it
                                          # regardless of how it is displayed
        self.prob_of_impulse_purchase = -1 # This is in [0,1] and indicates the proabability that a customer would impulsively purchase a product
                                      # TODO add more details, but this must be scaled significantly to be meaningful
        self.slots_per_display = -1 # This represents the amount of space each display of a product takes up
        self.min_display = -1 # This represents the minimum number of displays that can be put up of a product
                         # This will likely be 0, but accomodates the case that a brand refuses to have a small display in the store
        self.max_display = -1 # This represents the maximum number of displays that can be put up of a product
                         # This for example would represent the case where stock is limited
        self.units_per_display = -1 # This represents how many units of a product are in its display
        self.price_to_display_on_shelf_i_not_on_high_traffic = [] # price_to_display_on_shelf_i_not_on_high_traffic[i - 1] represens how much a brand is willing to pay to have their product placed on shelf level i while not on high traffic route
        self.price_to_display_on_shelf_i_on_high_traffic = [] # price_to_display_on_shelf_i_on_high_traffic[i - 1] represens how much a brand is willing to pay to have their product placed on shelf level i while on a high traffic route
        self.guirobi_var_is_on_shelf_i_level_j = []

    def get_name(self):
        return self.product_name
    
    # Guirobi discourages the use of spaces in variables names, so this removes them, see: https://docs.gurobi.com/projects/optimizer/en/13.0/reference/python/model.html
    def get_safe_name(self):
        return self.product_name.replace(" ", "_")

    def get_safe_category(self):
        return self.product_category.replace(" ", "_")

    def get_safe_shelf_type(self):
        return self.shelf_type.replace(" ", "_")

    def get_is_essential_item(self):
        return self.is_essential_item

    def get_slots_per_display(self):
        return self.slots_per_display
    
    def get_max_display(self):
        return self.max_display

    def get_min_display(self):
        return self.min_display

    def get_units_per_display(self):
        return self.units_per_display

    def get_profit_per_unit(self):
        return self.profit_per_unit
        
    def get_prob_of_intentional_purchase(self):
        return self.prob_of_intentional_purchase
        
    def print_product(self):
        print(f"product_name: {self.product_name}")
        print(f"product_category: {self.product_category}")
        print(f"shelf_type: {self.shelf_type}")
        print(f"is_essential_item: {self.is_essential_item}")
        print(f"profit_per_unit: {self.profit_per_unit}")
        print(f"prob_of_intentional_purchase: {self.prob_of_intentional_purchase}")
        print(f"prob_of_impulse_purchase: {self.prob_of_impulse_purchase}")
        print(f"slots_per_display: {self.slots_per_display}")
        print(f"min_display: {self.min_display}")
        print(f"max_display: {self.max_display}")
        print(f"units_per_display: {self.units_per_display}")
        for i in range(len(self.price_to_display_on_shelf_i_not_on_high_traffic)):
            print(f"price_to_display_on_shelf_i_not_on_high_traffic[{i}]: {self.price_to_display_on_shelf_i_not_on_high_traffic[i]}")
        for i in range(len(self.price_to_display_on_shelf_i_on_high_traffic)):
            print(f"price_to_display_on_shelf_i_on_high_traffic[{i}]: {self.price_to_display_on_shelf_i_on_high_traffic[i]}")
        
    def set_product_name(self, new_product_name):
        retVal = True
        new_product_name_str = ""
        try:
            new_product_name_str = str(new_product_name)
        except:
            print(f"An error occurred while calling set_product_name, unable to convert passed to str")
            retVal = False
            return retVal
        if new_product_name_str == "":
            print("Potential error occured when set_product_name was called, given string was empty")
            retVal = False
        self.product_name = new_product_name_str
        
        return retVal

    def set_shelf_type(self, shelf_type):
        retVal = True
        shelf_type_str = ""
        try:
            shelf_type_str = str(shelf_type)
        except:
            print(f"An error occurred while calling set_shelf_type, unavle to convert passed shelf_type to str")
            retVal = False
            return retVal
        if shelf_type_str == "":
            print("Error occured when set_shelf_type was called, given string was empty")
            retVal = False
        self.shelf_type = shelf_type_str
        return retVal

    def set_is_essential_item(self, is_essential):
        retVal = True
        is_essential_bool = False
        try:
            is_essential_bool = bool(is_essential)
        except:
            print(f"An error occurred while calling set_is_essential_item, unavle to convert passed is_essential to bool")
            retVal = False
            return retVal
        self.is_essential_item = is_essential_bool
        return retVal
        
    
    def set_product_category(self, product_category):
        retVal = True
        product_category_str = ""
        try:
            product_category_str = str(product_category)
        except:
            print(f"An error occurred while calling set_product_category, unable to convert passed to str")
            retVal = False
            return retVal
        if product_category_str == "":
            print("Error occured when set_product_category was called, given string was empty")
            retVal = False
        self.product_category = product_category_str
        
        return retVal
        

    def set_profit_per_unit(self, profit_per_unit):
        retVal  = True
        float_set_profit_per_unit = -1
        try:
            float_set_profit_per_unit = float(profit_per_unit)
        except:
            print(f"An error occurred while calling set_profit_per_unit for {self.product_name}, unable to convert profit_per_unit to float, recieved: {profit_per_unit}")
            retVal = False
            return retVal
        if float_set_profit_per_unit < 0: # Note: technically we could allow this for the case where the manufacturer pays lots for a display
            print(f"An error occurred, profit for product {self.product_name} was less than 0, this is not allowed, as in this case the product should not be stocked")
            retVal = False
        self.profit_per_unit = float_set_profit_per_unit
        return retVal

    def set_prob_of_intentional_purchase(self, prob_of_intentional_purchase):
        retVal = True
        float_prob_of_intentional_purchase = -1
        try:
            float_prob_of_intentional_purchase = float(prob_of_intentional_purchase)
        except:
            print(f"An error occurred while calling set_prob_of_intentional_purchase for {self.product_name}, unable to convert prob_of_intentional_purchase to float, recieved: {prob_of_intentional_purchase}")
            retVal = False
            return retVal
        if float_prob_of_intentional_purchase < 0 or float_prob_of_intentional_purchase > 1:
            print(f"An error occurred, prob of intentional purchase for product {self.product_name} was not in [0,1]")
            retVal = False
        self.prob_of_intentional_purchase = float_prob_of_intentional_purchase
        return retVal
        
    def set_prob_of_impulse_purchase(self, prob_of_impulse_purchase):
        retVal = True
        float_prob_of_impulse_purchase = -1
        try:
            float_prob_of_impulse_purchase = float(prob_of_impulse_purchase)
        except:
            print(f"An error occurred while calling set_prob_of_impulse_purchase for {self.product_name}, unable to convert prob_of_impulse_purchase to float, recieved: {prob_of_impulse_purchase}")
            retVal = False
            return retVal
        if float_prob_of_impulse_purchase < 0 or float_prob_of_impulse_purchase > 1:
            print(f"An error occurred, prob of impulse purchase for product {self.product_name} was not in [0,1]")
            retVal = False
        self.prob_of_impulse_purchase = float_prob_of_impulse_purchase
        return retVal
    
    def set_slots_per_display(self, slots_per_display):
        retVal = True
        int_slots_per_display = -1
        try:
            int_slots_per_display = int(slots_per_display)
        except:
            print(f"An error occurred while calling set_slots_per_display for {self.product_name}, unable to convert slots_per_display to an integer, recieved: {slots_per_display}")
            retVal = False
            return retVal
        if int_slots_per_display <= 0:
            printf(f"An error occured, value of slots_per_display for {self.product_name} is non-positive, this is not allowed each products display must take up atleast 1 space")
            retVal = False
        self.slots_per_display = int_slots_per_display
        return retVal
        
    def set_min_display(self, min_display):
        retVal = True
        int_min_display = -1
        try:
            int_min_display = int(min_display)
        except:
            print(f"An error occurred while calling set_min_display for {self.product_name}, unable to convert min_display to an integer, recieved: {min_display}")
            retVal = False
            return retVal
        if int_min_display < 0:
            printf(f"An error occured, value of min_display for {self.product_name} is negative")
            retVal = False
        self.min_display = int_min_display
        return retVal

    def set_max_display(self, max_display):
        retVal = True
        int_max_display = -1
        try:
            int_max_display = int(max_display)
        except:
            print(f"An error occurred while calling set_max_display for {self.product_name}, unable to convert max_display to an integer, recieved: {max_display}")
            retVal = False
            return retVal
        if int_max_display < 0: # Allowing 0 is allowed here, could consider banning it, if 0 we would not be stocking the product
            printf(f"An error occured, value of min_display for {self.product_name} is negative")
            retVal = False
        self.max_display = int_max_display
        return retVal

    def set_units_per_display(self, units_per_display):
        retVal = True
        int_units_per_display = -1
        try:
            int_units_per_display = int(units_per_display)
        except:
            print(f"An error occurred while calling set_units_per_display for {self.product_name}, unable to convert units_per_display to an integer, recieved: {units_per_display}")
            retVal = False
            return retVal
        if int_units_per_display <= 0: 
            printf(f"An error occured, value of units_per_display for {self.product_name} is non-positive") # A display must have atleast one product in it
            retVal = False
        self.units_per_display = int_units_per_display
        return retVal

    def set_price_to_display_on_shelf_i_not_on_high_traffic(self, arr):
        retVal = True
        if not isinstance(arr, list):
            print(f"Error occurred while calling set_price_to_display_on_shelf_i_not_on_high_traffic, the parameter passed was not a list")
            retVal = False
            return retVal
        if len(arr) != NUMBER_LEVELS_PER_SHELF:
            print(f"Error occurred while calling set_price_to_display_on_shelf_i_not_on_high_traffic, arr does not contain exactly {NUMBER_LEVELS_PER_SHELF} units, it contains {len(arr)}")
            retVal = False
            return retVal
        for display_price in arr:
            float_display_price = -1
            try:
                float_display_price = float(display_price)
            except:
                print(f"Error occurred while calling set_price_to_display_on_shelf_i_not_on_high_traffic, unable to covert an element to a float, recieved {display_price}")
            if float_display_price < 0:
                print(f"Error occurred while calling set_price_to_display_on_shelf_i_not_on_high_traffic, arr does contains a negative element")
                retVal = False
                return retVal 
            self.price_to_display_on_shelf_i_not_on_high_traffic.append(float_display_price)
        return retVal

    def set_price_to_display_on_shelf_i_on_high_traffic(self, arr):
        retVal = True
        if not isinstance(arr, list):
            print(f"Error occurred while calling set_price_to_display_on_shelf_i_on_high_traffic, the parameter passed was not a list")
            retVal = False
            return retVal
        if len(arr) != NUMBER_LEVELS_PER_SHELF:
            print(f"Error occurred while calling set_price_to_display_on_shelf_i_on_high_traffic, arr does not contain exactly {NUMBER_LEVELS_PER_SHELF} units, it contains {len(arr)}")
            retVal = False
            return retVal
        for display_price in arr:
            float_display_price = -1
            try:
                float_display_price = float(display_price)
            except:
                print(f"Error occurred while calling set_price_to_display_on_shelf_i_on_high_traffic, unable to covert an element to a float, recieved {display_price}")
            if float_display_price < 0:
                print(f"Error occurred while calling set_price_to_display_on_shelf_i_on_high_traffic, arr does contains a negative element")
                retVal = False
                return retVal
            self.price_to_display_on_shelf_i_on_high_traffic.append(float_display_price)
        return retVal

    # This is called to verify the product is properly initialized
    def isInitialized(self):
        retVal = True
        if not isinstance(self.product_name, str):
            print("product_name is not initialized to be a string")
            retVal = False
        elif self.product_name == "":
            print("product_name is the empty string string")
            retVal = False
        if not isinstance(self.profit_per_unit, float):
            print("profit_per_unit is not initialized to be a float")
            retVal = False
        elif self.profit_per_unit < 0:
            print("profit_per_unit is negative")
            retVal = False
        if not isinstance(self.prob_of_intentional_purchase, float):
            print("prob_of_intentional_purchase is not initialized to be a float")
            retVal = False
        elif self.prob_of_intentional_purchase < 0 or self.prob_of_intentional_purchase > 1:
            print("prob_of_intentional_purchase is not in [0,1]")
            retVal = False
        if not isinstance(self.slots_per_display, int):
            print("slots_per_display is not initialized to be a int")
            retVal = False
        if self.slots_per_display < 1:
            print("slots_per_display per display is non-positive")
            retVal = False
        if not isinstance(self.min_display, int):
            print("min_display is not initialized to be a int")
            retVal = False
        elif self.min_display < 0:
            print("min_display is negative")
            retVal = False
        if not isinstance(self.max_display, int):
            print("max_display is not initialized to be a int")
            retVal = False
        elif self.max_display < 1:
            print("max_display is non-positive")
            retVal = False
        if isinstance(self.min_display, int) and isinstance(self.max_display, int) and self.min_display > self.max_display:
            print("min_display is greater than max_display")
            retVal = False
        if not isinstance(self.units_per_display, int):
            print("units_per_display is not initialized to be a int")
            retVal = False
        elif self.units_per_display < 1:
            print("units_per_display is non-positive")
            retVal = False
        if not isinstance(self.price_to_display_on_shelf_i_not_on_high_traffic, list):
            print("price_to_display_on_shelf_i_not_on_high_traffic is not an instance of list")
            retVal = False
        elif len(self.price_to_display_on_shelf_i_not_on_high_traffic) != NUMBER_LEVELS_PER_SHELF:
            print("price_to_display_on_shelf_i_not_on_high_traffic does not have length NUMBER_LEVELS_PER_SHELF")
            retVal = False
        else:
            for display_price in self.price_to_display_on_shelf_i_not_on_high_traffic:
                if not isinstance(display_price, float):
                    print("A value of price_to_display_on_shelf_i_not_on_high_traffic is not a float")
                    retVal = False
                elif display_price < 0:
                    print("A value of price_to_display_on_shelf_i_not_on_high_traffic is negative")
                    retVal = False
        if not isinstance(self.price_to_display_on_shelf_i_on_high_traffic, list):
            print("price_to_display_on_shelf_i_on_high_traffic is not an instance of list")
            retVal = False
        elif len(self.price_to_display_on_shelf_i_on_high_traffic) != NUMBER_LEVELS_PER_SHELF:
            print("price_to_display_on_shelf_i_on_high_traffic does not have length NUMBER_LEVELS_PER_SHELF")
            retVal = False
        else:
            for display_price in self.price_to_display_on_shelf_i_on_high_traffic:
                if not isinstance(display_price, float):
                    print("A value of price_to_display_on_shelf_i_on_high_traffic is not a float")
                    retVal = False
                elif display_price < 0:
                    print("A value of price_to_display_on_shelf_i_on_high_traffic is negative")
                    retVal = False
        return retVal

In [6]:
# This defines a shelf
# TODO: Clean it up to look like product
class shelf:
    def __init__(self, shelf_id, shelf_type):
        self.shelf_id = shelf_id
        self.shelf_type = shelf_type

    def get_shelf_id(self):
        return self.shelf_id
    
    def get_shelf_type(self):
        return self.shelf_type.replace(' ', '_')

    def print(self):
        print(f"shelf_id: {self.shelf_id}")
        print(f"shelf_type: {self.shelf_type}")

In [7]:
# Set by python script
products = [] # This is made up of product class items, one for every product
shelves = []

In [8]:
# This script create a product class for every item found in the source_csv
# Requires the CSV file to be in the following format:
# Product Name, Category, profit per unit, purchase probability, impulse probability, Space per display, items per display, minimum number of displays, maximum number of displays, price willing to pay for shelf placement on level i not on high traffic, price willing to pay for shelf placement on level i on high traffic (For all i in 1 to NUMBER_LEVELS_PER_SHELF)
# Note, the first line of file is not read, as this is likely to be the key for the table
def initialize_products(source_csv):
    csv_data = pd.read_csv(source_csv)
    number_rows = csv_data.shape[0]
    for i in range(0, number_rows): 
        success = True
        product_object = product()
        product_name = csv_data.iloc[i,0]
        success = product_object.set_product_name(product_name) and success
        success = product_object.set_product_category(csv_data.iloc[i,1]) and success
        success = product_object.set_profit_per_unit(csv_data.iloc[i,2]) and success
        success = product_object.set_prob_of_intentional_purchase(csv_data.iloc[i,3]) and success
        success = product_object.set_prob_of_impulse_purchase(csv_data.iloc[i,4]) and success
        success = product_object.set_slots_per_display(csv_data.iloc[i,5]) and success
        success = product_object.set_units_per_display(csv_data.iloc[i,6]) and success
        success = product_object.set_min_display(csv_data.iloc[i,7]) and success
        success = product_object.set_max_display(csv_data.iloc[i,8]) and success

        price_arr_for_none_high_traffic = []
        price_arr_for_none_high_traffic.clear()
        price_arr_for_high_traffic = []
        price_arr_for_high_traffic.clear()
        for k in range(2 * NUMBER_LEVELS_PER_SHELF):
            if k % 2 == 0: # ie k is even
                price_arr_for_none_high_traffic.append(csv_data.iloc[i,9+k])
            else:
                price_arr_for_high_traffic.append(csv_data.iloc[i,9+k])
        success = product_object.set_price_to_display_on_shelf_i_not_on_high_traffic(price_arr_for_none_high_traffic) and success
        success = product_object.set_price_to_display_on_shelf_i_on_high_traffic(price_arr_for_high_traffic) and success

        if not success:
            print("An error occurred whilst initializating the product, it is printed below")
            product_object.print_product()
            return
        if not product_object.isInitialized():
            print("The product was not properly initialized, it is printed below")
            product_object.print_product()
            return
        products.append(product_object) 

# This is for for debugging, w
def print_all_products():
    for product in products:
        product.print_product()

# Given that the names of products are used in variable names, it is essentially all products have distinct names, this checks that
# Heavily inspired by https://stackoverflow.com/questions/1541797/how-do-i-check-if-there-are-duplicates-in-a-flat-list
def check_for_name_duplicates():
    name_list = []
    for product in products:
        name_list.append(product)
    seen = set()
    for product in products:
        product_name = product.get_safe_name()
        if product_name in seen:
            print(f"Error, product name {product_name} is used multiple times")
            return 
        seen.add(product_name)    

# Finds a product in product array, or prints an error
def find_in_product_array(product_name):
    for product in products:
        if product_name == product.get_name():
            return product
    print(f"ERROR: unable to find {product_name}")

# Processes supplemental data, that being if a product is an essential item and what shelf type a product must be assigned to
# TODO detail format
def process_supplemental_product_info(source_csv):
    csv_data = pd.read_csv(source_csv)
    number_rows = csv_data.shape[0]
    for i in range(0, number_rows):
        product_name = csv_data.iloc[i,0]
        shelf_type = csv_data.iloc[i,1]
        is_essential = False
        if csv_data.iloc[i,2] == 1:
            is_essential = True
        product = find_in_product_array(product_name)
        product.set_product_name(product_name)
        product.set_shelf_type(shelf_type)
        product.set_is_essential_item(is_essential)

# Process shelf data CSV
# TODO explain format
def proccess_shelf_info(source_csv):
    csv_data = pd.read_csv(source_csv)
    number_rows = csv_data.shape[0]
    for i in range(0, number_rows):
        shelf_id = csv_data.iloc[i,0]
        shelf_type = csv_data.iloc[i,1]
        shelves.append(shelf(shelf_id, shelf_type))

In [9]:
# Initialize products array
initialize_products(PRODUCT_CSV)
process_supplemental_product_info(SUPPLEMENT_PRODUCT_CSV)
proccess_shelf_info(SHELF_CSV)

# Verify initialization
# print_all_products() #Only want this if you are debugging
check_for_name_duplicates()
shelves_set = [] # Contains all IDs of the shelves
for shelf in shelves:
    shelves_set.append(shelf.get_shelf_id())
NUMBER_OF_SHELVES = len(shelves_set)


shelf_levels = list(range(1, NUMBER_LEVELS_PER_SHELF + 1)) # This simply contains all of the levels of each shelf

In [10]:
# Initialize the model
model = Model("RetailSpace")

Set parameter Username
Set parameter LicenseID to value 2709513
Academic license - for non-commercial use only - expires 2026-09-16


In [11]:
# Define variables that determine if product p is place on shelf b on level j
# is_product_i_on_shelf_j_level_k = model.addVars(product_name_list, range(1, NUMBER_OF_SHELVES), range(1, NUMBER_LEVELS_PER_SHELF), vtype=GRB.BINARY, name="is_product_i_on_shelf_j_level_k")

# Ensure that each product is assigned to atmost one shelf 
# (3.5.1.2)
# model.addConstrs((is_product_i_on_shelf_j_level_k.sum(product, '*') <= 1 for product in product_name_list), name = "Ensure each product is on atmost on slot")

# Define variables that determine what shelf level a product is on (variables is 1 if product is on that shelf level, 0 otherwise)
# is_product_i_on_level_j = model.addVars(product_name_list, range(1, NUMBER_LEVELS_PER_SHELF), vtype=GRB.BINARY, name = "is_product_i_on_level_j")
# Define a constraint forcing above definition to be true, this is simply summing over all shelves of is_product_i_on_shelf_j_level_k for a product
# (3.5.1.5)
# model.addConstrs((is_product_i_on_level_j[product,level] == is_product_i_on_shelf_j_level_k.sum(product, '*', level) for product in product_name_list for level in range(1, NUMBER_LEVELS_PER_SHELF)), name = "Calculate shelf level")


In [12]:
# List of product names
product_name_list = []
for product in products:
    product_name_list.append(product.get_safe_name())

# List of Categories
categories = []
for product in products:
    if product.get_safe_category() not in categories:
        categories.append(product.get_safe_category())

# List of shelf types
shelf_types = []
for product in products:
    if product.get_safe_shelf_type() not in shelf_types:
        shelf_types.append(product.get_safe_shelf_type())

# List of essential items
essential_items = []
for product in products:
    if product.get_is_essential_item():
        essential_items.append(product.get_safe_name())

In [13]:
# Variables

# This is x in the proposal
# This is set equal to 1 if product i is on shelf j level k and 0 otherwise
is_product_i_on_shelf_j_level_k = model.addVars(product_name_list, shelves_set, shelf_levels, vtype=GRB.BINARY, name="is_product_i_on_shelf_j_level_k")

# This is l in the proposal
# This is set to 1 if product i is on shelf level j and 0 otherwise
is_product_i_on_shelf_level_j = model.addVars(product_name_list, shelf_levels, vtype=GRB.BINARY, name="is_product_i_on_shelf_level_j")

# This is l' in the proposal
# This is set to 1 if product i is on shevel level j and that shelf is not on the high traffic route
is_product_i_on_shelf_level_j_with_shelf_not_on_high_traffic_route = model.addVars(product_name_list, shelf_levels, vtype=GRB.BINARY, name="is_product_i_on_shelf_level_j_with_shelf_not_on_high_traffic_route")

# This is l'' in the proposal
# This is set to 1 if product i is on shevel level j and that shelf is on the high traffic route
is_product_i_on_shelf_level_j_with_shelf_on_high_traffic_route = model.addVars(product_name_list, shelf_levels, vtype=GRB.BINARY, name="is_product_i_on_shelf_level_j_with_shelf_on_high_traffic_route")

# TODO DELETE: I am removing m as I am assuming they'll be charged, as there'd be no reason not to

# This is y in the proposal
# This is an non-negative integer denoting how many displays are assigned to product i
displays_assigned_to_product_i = model.addVars(product_name_list, vtype=GRB.INTEGER, lb=0, name="displays_assigned_to_product_i")

# This is z in the proposal
# This is 1 if product i is stocked by the store
is_product_i_stocked = model.addVars(product_name_list, vtype=GRB.BINARY, name="is_product_i_stocked")

# This is e in the proposal
# This is defined to be number of displays of product i on shelf j level k
displays_of_product_i_on_shelf_j_level_k = model.addVars(product_name_list, shelves_set, shelf_levels, vtype=GRB.INTEGER, lb=0, name="displays_of_product_i_on_shelf_j_level_k")

# TODO DELETE: I am removing f as it looks like g does its job

# This is g in the proposal
# This is the number of units of product on shelf i level j
total_units_on_shelf_i_level_j = model.addVars(shelves_set, shelf_levels, vtype=GRB.INTEGER, lb=0, name="total_units_on_shelf_i_level_j")

# This is h in the proposal
# This is 1 if category i is on shelf j
is_category_i_on_shelf_j = model.addVars(categories, shelves_set, vtype=GRB.BINARY, name="is_category_i_on_shelf_j")

# This is a in the proposal
# This is the total profit the store made by selling product i
profit_made_by_selling_i = model.addVars(product_name_list, vtype=GRB.CONTINUOUS, name="profit_made_by_selling_i")

# This is b in the proposal
# This is the total profit the store made from intentional demand of product i
profit_from_intentional_demand_by_selling_i = model.addVars(product_name_list, vtype=GRB.CONTINUOUS, name="profit_from_intentional_demand_by_selling_i")

# This is c in the proposal
# This is the total profit the store made from impulse demand of product i
profit_from_impulse_demand_by_selling_i = model.addVars(product_name_list, vtype=GRB.CONTINUOUS, name="profit_from_impulse_demand_by_selling_i")

# This is d in the proposal
# This is the total profit the store made from slotting fee of product i
profit_from_slotting_fees_for_i = model.addVars(product_name_list, vtype=GRB.CONTINUOUS, name="profit_from_slotting_fees_for_i")

In [14]:
# Create constraints

In [15]:
# This is (3.5.1.2) in the proposal (slightly modified as what was in there was not linear)
# Ensure that shelf i level j contains no more than NUMBER_SLOTS_PER_SHELF slots of product
for shelf in shelves_set:
    for level in shelf_levels:
        expr = LinExpr()
        for product in products:
            expr.add(displays_of_product_i_on_shelf_j_level_k[product.get_safe_name(), shelf, level], product.get_slots_per_display())
        model.addConstr(expr <= NUMBER_SLOTS_PER_SHELF, name =f"Ensure shelf {shelf} level {level} contains at most NUMBER_SLOTS_PER_SHELF slots of product")

In [16]:
# This is (3.5.1.3) it ensures that each product is assigned to ATMOST one shelf and level
model.addConstrs((is_product_i_on_shelf_j_level_k.sum(product, '*') <= 1 for product in product_name_list), name="Ensure that product i is assigned to atmost one shelf");

In [17]:
# This is (3.5.1.4) it set is_product_i_stocked to 1 iff the store stocks product i
model.addConstrs((is_product_i_on_shelf_j_level_k.sum(product, '*') == is_product_i_stocked[product] for product in product_name_list), name="Set is_product_i_stocked to 1 if product i is stocked");

In [18]:
# This is (3.5.1.5) in the proposal
# This sets is_product_i_on_shelf_level_j to 1 if product i is on shelf level j
model.addConstrs((is_product_i_on_shelf_j_level_k.sum(product, '*', level) == is_product_i_on_shelf_level_j[product, level] for product in product_name_list for level in shelf_levels), name="Set is_product_i_on_shelf_level_j to 1 if product i is on shelf level j");

In [19]:
# This is (3.5.2.3) in the proposal
# This ensures that displays_assigned_to_product_i is at most max_display
model.addConstrs((displays_assigned_to_product_i[product.get_safe_name()] <= product.get_max_display() for product in products), name="Ensures that displays_assigned_to_product_i is at most max_display");

In [20]:
# This is (3.5.2.4)
# This ensures that if product i is stocked, that at least min_display quantity is stocked
model.addConstrs((displays_assigned_to_product_i[product.get_safe_name()] >= is_product_i_stocked[product.get_safe_name()] * product.get_min_display() for product in products), name="This ensures that if product i is stocked, that at least min_display quantity is stocked");

In [21]:
# TODO DELETE (3.5.2.5) as it is redundant due to (3.5.1.2)

In [22]:
# This is (3.5.2.6)
# This ensures constraint on displays_of_product_i_on_shelf_j_level_k
model.addConstrs((displays_of_product_i_on_shelf_j_level_k[product, shelf, level] <= NUMBER_SLOTS_PER_SHELF *  is_product_i_on_shelf_j_level_k[product, shelf, level] for product in product_name_list for shelf in shelves_set for level in shelf_levels), name="Ensure that displays_of_product_i_on_shelf_j_level_k is set correctly");

In [23]:
# This is (3.5.2.7)
# This ensures displays_assigned_to_product_i is set correctly
model.addConstrs((displays_of_product_i_on_shelf_j_level_k.sum(product, '*') == displays_assigned_to_product_i[product] for product in product_name_list), name="Ensure displays_assigned_to_product_i is set correctly");

In [24]:
# TODO DELETE, I AM REMOVING (3.5.2.8) and (3.5.2.9), these seem redundant

In [25]:
# This is (3.5.3.3)
# This ensures that atmost one category is assigned to a shelf
model.addConstrs((is_category_i_on_shelf_j.sum('*', shelf) <= 1 for shelf in shelves_set), name="This ensures at most one category is assigned to a shelf");

In [26]:

# TODO DELETE, I AM REMOVING (3.5.3.2) it seems to ensure a category is only assigned to one shelf, but we want to allow several categories per shelf

In [27]:
# This is (3.5.3.4)
# This ensures that products are only assigned to a shelf if the shelf is assigned to its category
for category in categories:
    for shelf in shelves_set:
        expr = LinExpr()
        for product in products:
            if product.get_safe_category() == category:
                for level in shelf_levels:
                    expr.add(is_product_i_on_shelf_j_level_k[product.get_safe_name(), shelf, level]) 
        model.addConstr(expr <= NUMBER_LEVELS_PER_SHELF * NUMBER_SLOTS_PER_SHELF * is_category_i_on_shelf_j[category, shelf], name = f"Ensure {category} category products are on shelf {shelf} if the shelf is assigned to the category")
        # Note here that as each display of an item takes up atleast a slot, the most products a shelf could hold is NUMBER_LEVELS_PER_SHELF * NUMBER_SLOTS_PER_SHELF
        

In [28]:
# This is (3.5.4.*)
# It ensures that a product cannot be assigned to a shelf type it is not assigned to
for shelf in shelves:
    expr = LinExpr()
    for product in products:
        if product.get_safe_shelf_type() != shelf.get_shelf_type():
            for level in shelf_levels:
                expr.add(is_product_i_on_shelf_j_level_k[product.get_safe_name(), shelf.get_shelf_id(), level])
    model.addConstr(expr == 0, name = f"Ensure that only products of type {shelf.get_shelf_type()} are on shelf {shelf.get_shelf_id()}")         

In [29]:
# This is (3.5.8.1)
# It ensures that the most profit we can make is the amount of items stocked
    
model.addConstrs((profit_made_by_selling_i[product.get_safe_name()] <= product.get_units_per_display() * product.get_profit_per_unit() * displays_assigned_to_product_i[product.get_safe_name()] for product in products), name=f"This ensures that the most profit we can make on {product.get_safe_name()} is the amount of items stocked");

In [30]:
# TODO remove this
# This is a quick and dirty (3.5.8.2) ignoring impulse purchases, just to get proof of concept
model.addConstrs((profit_made_by_selling_i[product.get_safe_name()] <=  product.get_profit_per_unit() * product.get_prob_of_intentional_purchase() * PREDICT_CUSTOMERS_PER_DAY for product in products), name=f"This ensures that the most profit we can make on {product.get_safe_name()} is the amount that;'d be purchased");

In [31]:
# A quick objective function
model.setObjective(profit_made_by_selling_i.sum('*'), GRB.MAXIMIZE)

In [32]:
model.update()

In [33]:
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 24087 rows, 47201 columns and 188503 nonzeros
Model fingerprint: 0x39ce1e03
Variable types: 504 continuous, 46697 integer (24346 binary)
Coefficient statistics:
  Matrix range     [7e-01, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-02, 5e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 23124 rows and 45308 columns
Presolve time: 0.34s
Presolved: 963 rows, 1893 columns, 5583 nonzeros
Found heuristic solution: objective 1.0741897
Variable types: 0 continuous, 1893 integer (1167 binary)

Root relaxation: objective 2.419320e+01, 46 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | 

In [34]:
model.write('mode9.lp')


In [35]:
print(model)
for v in model.getVars():
    if v.x != 0:
        print(v.varName, "=", v.x)

<gurobi.Model MIP instance RetailSpace: 24087 constrs, 47201 vars, Parameter changes: Username=(user-defined), LicenseID=2709513>
is_product_i_on_shelf_j_level_k[Asparagus,23,5] = 1.0
is_product_i_on_shelf_j_level_k[Banana,4,1] = 1.0
is_product_i_on_shelf_j_level_k[Eggplant,16,5] = 1.0
is_product_i_on_shelf_j_level_k[Herbal_Tea,24,5] = 1.0
is_product_i_on_shelf_j_level_k[Icecream,29,3] = 1.0
is_product_i_on_shelf_j_level_k[Lettuce,23,4] = 1.0
is_product_i_on_shelf_j_level_k[Popcorn,1,1] = 1.0
is_product_i_on_shelf_j_level_k[Powdered_Sugar,22,5] = 1.0
is_product_i_on_shelf_j_level_k[Pretzels,20,1] = 1.0
is_product_i_on_shelf_j_level_k[Soda,5,1] = 1.0
is_product_i_on_shelf_level_j[Asparagus,5] = 1.0
is_product_i_on_shelf_level_j[Banana,1] = 1.0
is_product_i_on_shelf_level_j[Eggplant,5] = 1.0
is_product_i_on_shelf_level_j[Herbal_Tea,5] = 1.0
is_product_i_on_shelf_level_j[Icecream,3] = 1.0
is_product_i_on_shelf_level_j[Lettuce,4] = 1.0
is_product_i_on_shelf_level_j[Popcorn,1] = 1.0
is_prod